In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Read and explore the data

In [2]:
pl = pd.read_csv("data/odds/pl/pl1819.csv")

In [3]:
pl.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,...,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
1,E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,...,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
3,E0,11/08/2018,Huddersfield,Chelsea,0,3,A,0,2,A,...,1.84,23,1.00,1.84,1.80,2.13,2.06,7.24,3.95,1.58
4,E0,11/08/2018,Newcastle,Tottenham,1,2,A,1,2,A,...,1.81,20,0.25,2.20,2.12,1.80,1.76,4.74,3.53,1.89


In [4]:
pl.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA',
       'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD',
       'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
       'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

Define functions you want to use later on

In [5]:
#Returns all data for a specific team
def filter_team(df,name):
    return df[(pl.HomeTeam == name) | (df.AwayTeam == name)]

def home_games(df,name):
    return df[pl.HomeTeam == name]

def away_games(df,name):
    return df[df.AwayTeam == name]

def recent_points(df,lim):
    home_points = []
    away_points = []
    
    date_filter = 0
    for index, row in df.iterrows():
        home_team = filter_team(df[:date_filter],row["HomeTeam"])[-lim:] #last lim games for home team
        away_team = filter_team(df[:date_filter],row["AwayTeam"])[-lim:] #last lim games for away team

        
        home_wins = ( len(home_team[(home_team.HomeTeam==row["HomeTeam"])&(home_team.FTR=="H")]) 
                    + len(home_team[(home_team.AwayTeam==row["HomeTeam"])&(home_team.FTR=="A")]) )
        home_draws = len(home_team[home_team.FTR=="D"])
        
        away_wins = ( len(away_team[(away_team.HomeTeam==row["AwayTeam"])&(away_team.FTR=="H")]) 
            + len(away_team[(away_team.AwayTeam==row["AwayTeam"])&(away_team.FTR=="A")]) )
        away_draws = len(away_team[away_team.FTR=="D"])
        
        """
        print("Hometeam " + str(row["HomeTeam"]))
        display(home_team)
        print("home wins " + str(home_wins))
        print("home draws " + str(home_draws))
        print("home points " + str((3*home_wins) + home_draws))
        
        print("Away" + str(row["AwayTeam"]))
        display(away_team)
        print("away wins " + str(away_wins))
        print("away draws " + str(away_draws))
        print("away points " + str((3*away_wins) + away_draws))
        """

        
        home_points.append(((3*home_wins) + home_draws))
        away_points.append(((3*away_wins) + away_draws))
        
        
        date_filter = date_filter + 1
        
    df["home_points"] = home_points
    df["away_points"] = away_points
    return df


def recent_shots(df,lim):
    home_target_shots = []
    #home_woodwork_shots = []
    
    away_target_shots = []
    #away_woodwork_shots = []
    
    date_filter = 0
    for index, row in df.iterrows():
        home_team = filter_team(df[:date_filter],row["HomeTeam"])[-lim:] #last lim games for home team
        away_team = filter_team(df[:date_filter],row["AwayTeam"])[-lim:] #last lim games for away team
        
        hts = ( home_team[home_team.HomeTeam==row["HomeTeam"]].HST.sum() 
                + home_team[home_team.AwayTeam==row["HomeTeam"]].AST.sum())
        
        ats = ( away_team[away_team.HomeTeam==row["AwayTeam"]].HST.sum() 
                + away_team[away_team.AwayTeam==row["AwayTeam"]].AST.sum())
        
        """
        htws = ( home_team[home_team.HomeTeam==row["HomeTeam"]].HHW.sum() 
        + home_team[home_team.AwayTeam==row["HomeTeam"]].AHW.sum())
        
        atws = ( away_team[away_team.HomeTeam==row["AwayTeam"]].HHW.sum() 
                + away_team[away_team.AwayTeam==row["AwayTeam"]].AHW.sum())
        """

        
        home_target_shots.append(hts)
        #home_woodwork_shots.append(htws)

        away_target_shots.append(ats)
        #away_woodwork_shots.append(atws)
        
        date_filter = date_filter + 1
    
    df["hts"] = home_target_shots
    #df["htws"] = home_woodwork_shots
    df["ats"] = away_target_shots
    #df["atws"] = away_woodwork_shots
    return df

    
def recent_corners(df,lim):
    home_corners = []
    away_corners = []

    date_filter = 0
    for index, row in df.iterrows():
        home_team = filter_team(df[:date_filter],row["HomeTeam"])[-lim:] #last lim games for home team
        away_team = filter_team(df[:date_filter],row["AwayTeam"])[-lim:] #last lim games for away team
        
        htc = ( home_team[home_team.HomeTeam==row["HomeTeam"]].HC.sum() 
                + home_team[home_team.AwayTeam==row["HomeTeam"]].AC.sum())
        
        atc = ( away_team[away_team.HomeTeam==row["AwayTeam"]].HC.sum() 
                + away_team[away_team.AwayTeam==row["AwayTeam"]].AC.sum())
                
        home_corners.append(htc)

        away_corners.append(atc)
        
        date_filter = date_filter + 1
    
    df["htc"] = home_corners
    df["atc"] = away_corners
    return df                                                                                                                                                                                                                                                            

In [6]:
filter_team(pl,"Fulham").head() #Example of a filter function (all Fulham matches)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
14,E0,18/08/2018,Tottenham,Fulham,3,1,H,1,0,H,...,2.54,21,-1.75,2.06,2.00,1.91,1.86,1.27,6.50,11.96
26,E0,26/08/2018,Fulham,Burnley,4,2,H,3,2,H,...,1.65,20,-0.25,1.74,1.69,2.32,2.22,2.26,3.23,3.71
30,E0,01/09/2018,Brighton,Fulham,2,2,D,0,1,A,...,1.88,20,-0.25,1.95,1.91,2.01,1.96,2.17,3.52,3.61
43,E0,15/09/2018,Man City,Fulham,3,0,H,2,0,H,...,3.85,23,-2.50,2.00,1.95,1.96,1.92,1.10,12.75,25.00


### Feature engineering
Select a set of features that may be important for the classification task. Read notes.txt for further description of each feature. We will in this case make predictions based on shots, points and corners in the previous four games.

In [7]:
df = pl[["FTR","HomeTeam","AwayTeam","HST","AST","HC","AC","B365H","B365D","B365A"]]
df = recent_points(df,4)
df = recent_shots(df,4)
df = recent_corners(df,4)
df = df[(len(df.HomeTeam.unique()) * 4 ):] #After the teams have played 4 games
df.drop(['HST', 'AST', 'HC', 'AC', 'HomeTeam', 'AwayTeam'], axis=1, inplace=True)
df = df.replace(to_replace=['H', 'D', 'A'], value=[0,1,2]) #Represent results as numbers

In [8]:
df.head()

,FTR,B365H,B365D,B365A,home_points,away_points,hts,ats,htc,atc
80,1,2.04,3.7,3.75,9,1,22,18,22,15
81,0,2.50,3.5,2.95,0,1,15,10,17,17
82,1,1.75,3.8,5.25,8,7,23,26,29,32
83,2,11.00,5.5,1.33,1,8,11,22,19,15
84,0,1.08,13.0,34.00,10,7,29,12,35,8


### Classifier

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
df = shuffle(df)
y = df.FTR #Target values (home,away,draw)
X = df.drop(['FTR'], axis=1)

In [10]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=20)
scores = cross_val_score(clf, X, y, cv=10)
print("Scores: " + str(scores))
print("Average: " + str(sum(scores)/len(scores) ))

Scores: [0.51612903 0.61290323 0.67741935 0.51612903 0.58064516 0.5
 0.6        0.51724138 0.42857143 0.57142857]
Average: 0.5520467185761958


In [11]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
scores = cross_val_score(clf, X, y, cv=10)
print("Scores: " + str(scores))
print("Average: " + str(sum(scores)/len(scores) ))

Scores: [0.4516129  0.58064516 0.41935484 0.51612903 0.4516129  0.46666667
 0.33333333 0.5862069  0.39285714 0.32142857]
Average: 0.45198474495471164


In [12]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
scores = cross_val_score(clf, X, y, cv=10)
print("Scores: " + str(scores))
print("Average: " + str(sum(scores)/len(scores) ))

Scores: [0.58064516 0.58064516 0.58064516 0.61290323 0.51612903 0.56666667
 0.53333333 0.5862069  0.53571429 0.39285714]
Average: 0.5485746067058637


In [13]:
from sklearn.svm import SVC

clf = SVC(gamma=0.001)
scores = cross_val_score(clf, X, y, cv=10)
print("Scores: " + str(scores))
print("Average: " + str(sum(scores)/len(scores) ))

Scores: [0.64516129 0.61290323 0.61290323 0.51612903 0.64516129 0.56666667
 0.5        0.51724138 0.5        0.57142857]
Average: 0.5687594681921712


In [14]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X, y, cv=10)
print("Scores: " + str(scores))
print("Average: " + str(sum(scores)/len(scores) ))

Scores: [0.35483871 0.48387097 0.67741935 0.48387097 0.5483871  0.33333333
 0.43333333 0.44827586 0.5        0.5       ]
Average: 0.47633296255098256


In the end, save your favourite model

In [15]:
from joblib import dump,load

clf = SVC(gamma=0.001)
clf.fit(X,y)

dump(clf,"clf.joblib")

['clf.joblib']

Use your model to predict games

In [16]:
model = load('clf.joblib')

In [17]:
game = df[1:10]
game #FTR is the target label

,FTR,B365H,B365D,B365A,home_points,away_points,hts,ats,htc,atc
288,0,2.55,3.30,3.00,7,3,12,23,15,28
115,1,2.50,3.30,3.10,2,6,20,12,19,24
376,2,1.28,6.50,11.00,2,3,14,21,16,22
363,2,9.50,5.25,1.36,7,12,17,25,20,28
180,1,5.25,3.90,1.72,3,7,11,16,20,17
343,2,17.00,7.50,1.20,3,12,9,21,14,34
95,0,1.75,3.75,5.25,4,1,15,10,26,16
131,0,1.61,3.90,6.50,2,1,11,9,21,14
301,0,1.55,4.20,7.00,6,3,14,9,20,12


In [18]:
model.predict(game.drop(['FTR'], axis=1))

array([0, 0, 0, 2, 0, 2, 0, 0, 0])